Create deadlock.db

In [4]:
import duckdb
import pandas as pd

con = duckdb.connect("c:/Code/Local Code/Deadlock Database/Deadlock_Match_Prediction/deadlock.db")

Drop tables for reset

In [ ]:
def drop_tables(con):
    con.execute("DROP TABLE IF EXISTS matches")
    con.execute("DROP TABLE IF EXISTS player_matches")
    con.execute("DROP TABLE IF EXISTS player_trends")
    con.execute("DROP TABLE IF EXISTS hero_trends")

Create tables

In [ ]:
con.execute("""
CREATE TABLE matches (
    match_id BIGINT PRIMARY KEY,
    start_time TIMESTAMP,
    game_mode INTEGER,
    match_mode INTEGER,
    match_duration_s INTEGER,
    objectives_mask_team0 BIGINT,
    objectives_mask_team1 BIGINT,
    match_result VARCHAR
)
""")

In [ ]:
con.execute("""
CREATE TABLE player_matches (
    account_id BIGINT,
    match_id BIGINT,
    hero_id INTEGER,
    hero_level INTEGER,
    player_team INTEGER,
    player_kills INTEGER,
    player_deaths INTEGER,
    player_assists INTEGER,
    denies INTEGER,
    net_worth BIGINT,
    last_hits INTEGER,
    team_abandoned BOOLEAN,
    abandoned_time_s INTEGER,
    won BOOLEAN,
    
    PRIMARY KEY (account_id, match_id)
)
""")

In [ ]:
con.execute("""
CREATE TABLE player_trends (
    account_id BIGINT,
    match_id BIGINT,
    hero_id INTEGER,
    p_win_pct_3 FLOAT,
    p_win_pct_5 FLOAT,
    p_streak_3 VARCHAR,
    p_streak_5 VARCHAR,
    h_win_pct_3 FLOAT,
    h_win_pct_5 FLOAT,
    h_streak_3 VARCHAR,
    h_streak_5 VARCHAR,

    PRIMARY KEY (account_id, match_id, hero_id)
)
""")

In [ ]:
con.execute("""
CREATE TABLE hero_trends (
    hero_id INTEGER,
    trend_start_date DATE,
    trend_end_date DATE,
    trend_date DATE,
    trend_window_days INTEGER,
    pick_rate FLOAT,
    win_rate FLOAT,
    average_kills FLOAT,
    average_deaths FLOAT,
    average_assists FLOAT,

    PRIMARY KEY (hero_id, trend_start_date, trend_end_date, trend_window_days)
)
""")


In [ ]:
con.execute("""
    CREATE TABLE player_profiles (
    account_id BIGINT PRIMARY KEY,
    
    avg_p_win_pct_3 FLOAT,
    avg_p_win_pct_5 FLOAT,
    avg_h_win_pct_3 FLOAT,
    avg_h_win_pct_5 FLOAT,
    
    total_matches INTEGER,

    norm_avg_p_win_pct_3 FLOAT,  
    norm_avg_p_win_pct_5 FLOAT,
    norm_avg_h_win_pct_3 FLOAT,
    norm_avg_h_win_pct_5 FLOAT
)
""")

duckdb sql checks

In [8]:
con.execute("SELECT COUNT(*) FROM match_holdout").fetchone()[0]

5968

In [ ]:
result = con.execute("SELECT * FROM player_profiles LIMIT 5").fetchdf()
print(result)

In [ ]:
import requests
m_id = [471321,471609,471883]
site = "https://api.deadlock-api.com"
endpoint = "/v1/matches/metadata?"
url=f"{site}{endpoint}include_player_info=true&match_ids={m_id}"
response = requests.get(url)
if response.status_code == 200:
    json_match_data = response.json()
    df_match_data = pd.DataFrame(response.json())
else:
    print(f"\n\nFailed to retrieve match data: {response.status_code}\n\n response: {response}\n\n")
print(response.json)

In [ ]:
con.execute("""
            CREATE TABLE match_holdout AS
SELECT *
FROM match_players
WHERE match_id IN (
    SELECT match_id
    FROM match_players
    GROUP BY match_id
    ORDER BY RANDOM()
    LIMIT 5000  -- or however many you want to hold out
);""")